In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets evaluate --upgrade

## Data Pre-processing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

In [ ]:
hindi_text = "राजीव हत्याकांड: चेन्नई के घर में शिवरासन ने क्यों खोदा था तीन फुट का गड्ढा, CBI को उसमें "
tokens = tokenizer.tokenize(hindi_text)
print(tokens)

['▁रज', 'व', '▁हत', 'य', 'कड', ':', '▁', 'चन', 'नई', '▁क', '▁घर', '▁म', '▁शव', 'रसन', '▁न', '▁क', 'य', '▁खद', '▁थ', '▁तन', '▁फट', '▁क', '▁गड', 'ढ', ',', '▁c', 'bi', '▁क', '▁उस', 'म', '▁']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import torchtext
import datasets
import tqdm
import evaluate
import pandas as pd

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Task_3/team8_hi_train.csv', names=['en', 'hi'])
df_test = pd.read_csv('/content/drive/MyDrive/Task_3/team8_hi_test.csv', names=['en', 'hi'])
df_valid = pd.read_csv('/content/drive/MyDrive/Task_3/team8_hi_valid.csv', names=['en', 'hi'])
train_data,test_data,valid_data = [],[],[]
for _, row in df_train.iterrows(): train_data.append({'en': row['en'], 'hi': row['hi']})
for _, row in df_test.iterrows(): test_data.append({'en': row['en'], 'hi': row['hi']})
for _, row in df_valid.iterrows(): valid_data.append({'en': row['en'], 'hi': row['hi']})

In [ ]:
del test_data[0]
del train_data[0]
del valid_data[0]
train_data[0]

{'en': 'CAA is a separate law and NRC is a separate process.',
 'hi': 'सीएए अलग कानून है और एनआरसी एक अलग प्रक्रिया है।'}

In [ ]:
train_dataset = datasets.Dataset.from_list(train_data)
test_dataset = datasets.Dataset.from_list(test_data)
valid_dataset = datasets.Dataset.from_list(valid_data)

In [ ]:
def tokenize_example(example, max_length, sos_token, eos_token):
    en_tokens = [token for token in tokenizer.tokenize(example["en"])][:max_length]
    hi_tokens = [token for token in tokenizer.tokenize(example["hi"])][:max_length]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    hi_tokens = [sos_token] + hi_tokens + [eos_token]
    return {"en_tokens": en_tokens, "hi_tokens": hi_tokens}

In [ ]:
max_length = 1000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'en': 'CAA is a separate law and NRC is a separate process.',
 'hi': 'सीएए अलग कानून है और एनआरसी एक अलग प्रक्रिया है।',
 'en_tokens': ['<sos>',
  '▁ca',
  'a',
  '▁is',
  '▁a',
  '▁separate',
  '▁law',
  '▁and',
  '▁n',
  'rc',
  '▁is',
  '▁a',
  '▁separate',
  '▁process',
  '.',
  '<eos>'],
 'hi_tokens': ['<sos>',
  '▁स',
  'ए',
  'ए',
  '▁अलग',
  '▁कन',
  'न',
  '▁ह',
  '▁और',
  '▁एनआर',
  'स',
  '▁एक',
  '▁अलग',
  '▁पर',
  'कर',
  'य',
  '▁ह',
  '।',
  '<eos>']}

In [ ]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]



In [ ]:
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
en_vocab = build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

hi_vocab = build_vocab_from_iterator(
    train_data["hi_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [ ]:
en_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '▁the',
 '.',
 ',',
 '▁of',
 '▁',
 '▁and',
 '▁in',
 '▁to',
 '▁a',
 's',
 '▁is',
 '▁for',
 '"',
 'ed',
 '▁on',
 '▁that']

In [ ]:
hi_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '▁क', '▁ह', 'य', '▁म', 'त', '।']

In [ ]:
assert en_vocab[unk_token] == hi_vocab[unk_token]
assert en_vocab[pad_token] == hi_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [ ]:
en_vocab.set_default_index(unk_index)
hi_vocab.set_default_index(unk_index)

In [ ]:
def numericalize_example(example, en_vocab, hi_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    hi_ids = hi_vocab.lookup_indices(example["hi_tokens"])
    return {"en_ids": en_ids, "hi_ids": hi_ids}

In [ ]:
fn_kwargs = {"en_vocab": en_vocab, "hi_vocab": hi_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'en': 'CAA is a separate law and NRC is a separate process.',
 'hi': 'सीएए अलग कानून है और एनआरसी एक अलग प्रक्रिया है।',
 'en_tokens': ['<sos>',
  '▁ca',
  'a',
  '▁is',
  '▁a',
  '▁separate',
  '▁law',
  '▁and',
  '▁n',
  'rc',
  '▁is',
  '▁a',
  '▁separate',
  '▁process',
  '.',
  '<eos>'],
 'hi_tokens': ['<sos>',
  '▁स',
  'ए',
  'ए',
  '▁अलग',
  '▁कन',
  'न',
  '▁ह',
  '▁और',
  '▁एनआर',
  'स',
  '▁एक',
  '▁अलग',
  '▁पर',
  'कर',
  'य',
  '▁ह',
  '।',
  '<eos>'],
 'en_ids': [2,
  1704,
  72,
  14,
  12,
  1714,
  330,
  9,
  93,
  1695,
  14,
  12,
  1714,
  622,
  5,
  3],
 'hi_ids': [2,
  13,
  19,
  19,
  483,
  178,
  16,
  5,
  17,
  1318,
  18,
  47,
  483,
  10,
  32,
  6,
  5,
  9,
  3]}

In [ ]:
data_type = "torch"
format_columns = ["en_ids", "hi_ids"]


train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

## Data Loader

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_hi_ids = [example["hi_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_hi_ids = nn.utils.rnn.pad_sequence(batch_hi_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "hi_ids": batch_hi_ids,
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 1000
train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)


## Building the Encoder


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

## Building the Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

## Connecting Encoder and Decoder

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

## Model and Weight initialization

In [ ]:
input_dim = len(en_vocab)
output_dim = len(hi_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 1
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15476, 256)
    (rnn): LSTM(256, 512, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4707, 256)
    (rnn): LSTM(256, 512, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=4707, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 10,735,459 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

## Training the Model

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["en_ids"].to(device)
        trg = batch["hi_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["en_ids"].to(device)
            trg = batch["hi_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
n_epochs = 1
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|██████████| 1/1 [00:28<00:00, 28.52s/it]

	Train Loss:   5.860 | Train PPL: 350.600
	Valid Loss:   5.744 | Valid PPL: 312.166


## Evaluating the Model


In [ ]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

## Inferencing

In [ ]:
def translate_sentence(
    sentence,
    model,
    en_vocab,
    hi_vocab,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token for token in tokenizer.tokenize(sentence)]
        else:
            tokens = [token for token in sentence]

        tokens = [sos_token] + tokens + [eos_token]
        ids = en_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = hi_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == hi_vocab[eos_token]:
                break
        tokens = hi_vocab.lookup_tokens(inputs)
    return tokens

In [ ]:
sentence = test_data[0]["en"]
expected_translation = test_data[0]["hi"]

sentence, expected_translation

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    en_vocab,
    hi_vocab,
    sos_token,
    eos_token,
    device,
)

In [ ]:
translation

In [ ]:
sentence = "Why be the king, when you can be the god? "

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    en_vocab,
    hi_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [ ]:
translation

## Bleu Score

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
translations = [
    translate_sentence(
        example["en"],
        model,
        en_vocab,
        hi_vocab,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]

In [ ]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["hi"]] for example in test_data]

In [ ]:
predictions[0], references[0]

In [ ]:
def tokenizer_fn(s):
      tokens = [token for token in tokenizer.tokenize(s)]
      return tokens


In [ ]:
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

In [ ]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [ ]:
results